## Исследование надёжности заёмщиков

Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. 
Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

In [1]:
import pandas as pd

In [5]:
data = pd.read_csv('/home/drresist/Documents/Course projects/datasets/preprocessing_data.csv')
data.head()


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [6]:
data.tail()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

Нам дана таблица размернсотью (21525 х 12) со смешанными типами данных. 

Мы можем наблюдать как целочисленне значения, так и с плавающей точкой. 
Также, из таблицы видно, что в столбцах *days_employed & total_income* наблюдаются пропуски. Далее нам потребуется выяснить связаны ли эти пропуски и возможные причины их появления.

Значения в столбце "purpose", вероятно, заполняются клиентом/сотрудников, в результате чего одинаковая причина может иметь разную запись. (к примеру, "на покупку своего автомобиля", "на покупку автомобиля", и т.д.). Нам потребуется лемматизация для более точного разделения целей кредита на категории.

В столбце "days_employed" наблюдаются отрицательные значения, что не может быть правдой. Приведём их к абсолютному значению.

Исходя из выборки *data.head()* видно, что столбцы с строковыми значениями могут быть в разном регистре. При дальнейшей работе потребуется перевести их в нижний регистр. 



#### Пропущенные значения

Проведём проверку таблицы на пустые значения. Используем для этого функцию *.isna()*

In [8]:
print(data['days_employed'].isna().sum())
print(data['total_income'].isna().sum())


2174
2174


Проверим, совпадают ли эти люди.

In [9]:
data['days_employed'].isna().all() == data['total_income'].isna().all()

True

Мы видим, что люди "без зарплаты" также не имеют "опыта работы". Обе эти переменные являются количественными. Исправим данные значения присвоив им медианные значения, но тут есть проблема. Посмотрим внимательнее на столбцы:

In [10]:
print(data[data['days_employed']<0]['days_employed'].count()) # Кол-во отрицательных значений
print()
print(data['days_employed'].sort_values(ascending=False).head(3)) # Кол-во дней опыта
print()
print(data['days_employed'].sort_values(ascending=True).head(3)) # Кол-во дней опыта (отрицательный)

15906

6954     401755.400475
10006    401715.811749
7664     401675.093434
Name: days_employed, dtype: float64

16335   -18388.949901
4299    -17615.563266
7329    -16593.472817
Name: days_employed, dtype: float64


Некоторые клиенты имеют более 1000 лет опыта(!), что далеко от реальности, а у некоторых - есть отрицательный опыт. 
Для дальнейшей работы нам необходимо исправить данные записи. 
Избавиться от отрицательных значений возможно через функцию *.abs()*, которая возвращает абсолютное значение, но у нас в таблице остались *NaN* значения.
Заменим их на 0.

In [11]:
data['days_employed'] = data['days_employed'].fillna(0)
data['total_income'] = data['total_income'].fillna(0)

data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Мы избавились от пропусков. Теперь нужно привести таблицы к целочисленным не отрицательным значениям.

In [12]:
data['days_employed'] = data['days_employed'].abs()
data['total_income'] = data['total_income'].abs()

data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Вычислим среднее и медианное значение возраста клиентов:

In [13]:
median_dob_years = data['dob_years'].median()
mean_dob_years = data['dob_years'].mean()
print(f"Медиана = {median_dob_years}\nСреднее значение = {mean_dob_years}")


Медиана = 42.0
Среднее значение = 43.29337979094077


Среднее и медианное значение по зарплате в данный момент без корректировок:

In [14]:
median_days_employed = data['days_employed'].median()
mean_days_employed = data['days_employed'].mean()
print(median_days_employed)
print(mean_days_employed)

1808.0534339280625
60156.419004688105


Показатели примерно равны. Средний возраст клиентов колеблется между 42-43 годами. 

В случае опыта работы мы видим, что средний показатель и медианный сильно отличаются. Медианный приближен к реальности и примерно равен ~ 5 годам. 

Заменим нулевые значения возраста на средний показатель. 

In [15]:
data.loc[data['dob_years'] ==0, 'dob_years'] = median_dob_years

У нас осталась одна проблема с "опытом работы". Нам нужно привести слишком высокие значения к среднему, так как это может повлиять на дальнейший анализ.

Для выявления таких записей можно воспользоваться следующей логикой:
```
Если стаж в днях > (возраст-18) * 365 => запись некорректна. 
```

Реализуем это в виде кода:

In [16]:
data[data['days_employed']>data['dob_years']*365-18]['days_employed'].count()


3445

У нас имеется 3445 записей с некорректным опытом работы. Заменим некорректные значения на медианные.

In [14]:
data.loc[data['days_employed']>data['dob_years']*365-18, 'days_employed'] = median_days_employed

### Вывод

Из выборки видно, что люди без опыта работы и без указанного ежемесячного дохода - это одинаковые люди. 
Для дальнейшего построения гипотез и исключения влияния таких записей была произведена замена значений на медианные по столбцам. 

Причин для появления подобных записей без информации может быть несколько:

* Человек занимается предпринимательством, в связи с чем не имеет опыт работы и постоянного установленного заработка
* Человек не имеет опыт работы / постоянный заработок
* Отказ от заполнения данных
* Некорректно внесены данные в таблицу




### Замена типа данных

Посмотрим на типы данных в таблице после проведенных операций. 

In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  float64
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(3), int64(4), object(5)
memory usage: 2.0+ MB


В столбцах *days_employed*, *dob_years*, *total_income* у нас вещественный тип данных (*float64*). Для приведения к целочисленному типу данных используем функцию .astype(). Использование функции *.to_numeric()* не требуется, так как она используется для перевода строкового типа в вещественный. Наша задача - из вещественного в целочисленный.        

Сохраним столбцы, которые нам нужно перевести в целые числа, в отдельную переменную *columns*:

In [18]:
int_columns = ['days_employed', 'dob_years','total_income']

Применим функцию *.apply* к нашим столбцам. В самой функции укажем безымянную функцию (лямбда-функция), в которой происходит преобразование в целое число используя *.astype()*:

In [19]:
data[int_columns] = data[int_columns].apply(lambda x: x.astype(int))

In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

В таблице были найденны отрицательные значения, *float(числа с плавающей точкой)*, так и *int(целочисленные значения)*.
Для общего типа данных привели их к единому типу - целые числа. 
Для этого была использована конструкция *lambda* и функции *.apply()*, *.astype()*

### Обработка дубликатов

Проверим количество одинаковых значений в текущем состоянии таблицы.

In [21]:
data.duplicated().sum()

55

Сохраним строковые столбцы в переменной *str_columns* для упрощения дальнейшей работы с ними. 

In [22]:
str_columns  = ['education', 'family_status', 'gender', 'purpose']

У нас есть 55 продублированных записи. Приведём строковые значения таблицы к нижнему регистру и проверим количество "дублей" после. Для этого мы можем использовать функию *.apply()*. Для краткости мы можем использовать конструкцию **lambda** или же написать собственную функцию:

**Примечание**: Для дальнейшей работы можно использовать один из блоков ниже. Они обладают одинаковым функционалом. 

In [23]:
def lower(n):
    """
    param n: Принимаем строку
    return: Если запись str → возвращаем её в нижнем регистре
    """
    return n.astype(str).str.lower()
data[str_columns] = data[str_columns].apply(lower)

In [24]:
data[str_columns] = data[str_columns].apply(lambda st: st.astype(str).str.lower())

In [25]:
data.duplicated().sum()

72

У нас имеется 72 повторяющиеся записи. Удалим их для исключения влияния на итоговую выборку. Используем для этого функцию *.drop_duplicates()* + *.reset_index(drop=True)*. Вторая функция позволяет нам сбросить индексы и переназначить их заново. 

In [26]:
data.drop_duplicates().reset_index(drop=True)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21448,1,4529,43,среднее,1,гражданский брак,1,f,компаньон,0,224791,операции с жильем
21449,0,343937,67,среднее,1,женат / замужем,0,f,пенсионер,0,155999,сделка с автомобилем
21450,1,2113,38,среднее,1,гражданский брак,1,m,сотрудник,1,89672,недвижимость
21451,3,3112,38,среднее,1,женат / замужем,0,m,сотрудник,1,244093,на покупку своего автомобиля


### Вывод

До приведения строковых значений к нижнему регистру было выявлено 55 продублированных записей, что исходя из объёма таблицы (~ 22 тыс записей) не большое число. Однако, после приобразования всех строковых значений в нижний регистр было найдено 72 записей.

Возможной причиной появления таких записей может служить отправление клиентом нескольких заявок, в результате чего они были повторно внесены в базу. 

Далее, мы провели очистку таблицы от таких записей с последующим сбросом индексов. 

После полной очистки на выходе у нас имеется 21453 записей. 



### Лемматизация

Проведём лемматизация последнего столбца - *purpose*. В нем указана цель кредита. При помощи лемматизации мы сможем более явно проследить основные причины взятия кредита. 

Для лемматизации используем библиотеку *pymystem3* и её функцию *Mystem*. В дополнение, используем функцию *Counter* из библиотеки *collections* для подсчета повторяющихся слов в столбце. 

In [29]:
from pymystem3 import Mystem
from collections import Counter
m = Mystem()

Installing mystem to /home/drresist/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


Так как у нас большая таблица и нам не требуется анализировать причины взятия кредита по каждому клиенту, то выделим уникальные значения и подсчитаем их количество. 

In [30]:
purpose_list = list(data['purpose'].unique())
len(purpose_list)

38

У нас получилось 38 значений, но это не означает, что все причины можно поделить на 38 категории. Так как причина заполняется вручную, то одна и та же цель может быть выражена по разному. Рассмотрим первые 10 строк в таблице.

In [31]:
data['purpose'].head(10)

0                 покупка жилья
1       приобретение автомобиля
2                 покупка жилья
3    дополнительное образование
4               сыграть свадьбу
5                 покупка жилья
6             операции с жильем
7                   образование
8         на проведение свадьбы
9       покупка жилья для семьи
Name: purpose, dtype: object

Уже тут видно, что покупка жилья может интерпретироваться по разному:

* покупка жилья
* операции с жильем
* покупка жилья для семьи

Проведём лемматизацию для упрощения дальнейшей работы с причинами:

In [32]:
purpose_lemmas = [m.lemmatize(line) for line in purpose_list]
purpose_lemmas

[['покупка', ' ', 'жилье', '\n'],
 ['приобретение', ' ', 'автомобиль', '\n'],
 ['дополнительный', ' ', 'образование', '\n'],
 ['сыграть', ' ', 'свадьба', '\n'],
 ['операция', ' ', 'с', ' ', 'жилье', '\n'],
 ['образование', '\n'],
 ['на', ' ', 'проведение', ' ', 'свадьба', '\n'],
 ['покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', '\n'],
 ['покупка', ' ', 'недвижимость', '\n'],
 ['покупка', ' ', 'коммерческий', ' ', 'недвижимость', '\n'],
 ['покупка', ' ', 'жилой', ' ', 'недвижимость', '\n'],
 ['строительство', ' ', 'собственный', ' ', 'недвижимость', '\n'],
 ['недвижимость', '\n'],
 ['строительство', ' ', 'недвижимость', '\n'],
 ['на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', '\n'],
 ['на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', '\n'],
 ['операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', '\n'],
 ['строительство', ' ', 'жилой', ' ', 'недвижимость', '\n'],
 ['жилье', '\n'],
 ['операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', '\n'],
 ['автомобиль'

Мы получили *list of list*. Для подсчета часто встречающихся причин приведём это все в один список и применим функцию *Counter*:

In [33]:
flat_list = [item for sublist in purpose_lemmas for item in sublist]

Counter(flat_list).most_common()

[(' ', 59),
 ('\n', 38),
 ('покупка', 10),
 ('недвижимость', 10),
 ('автомобиль', 9),
 ('образование', 9),
 ('жилье', 7),
 ('с', 5),
 ('операция', 4),
 ('на', 4),
 ('свой', 4),
 ('свадьба', 3),
 ('строительство', 3),
 ('получение', 3),
 ('высокий', 3),
 ('дополнительный', 2),
 ('для', 2),
 ('коммерческий', 2),
 ('жилой', 2),
 ('заниматься', 2),
 ('сделка', 2),
 ('приобретение', 1),
 ('сыграть', 1),
 ('проведение', 1),
 ('семья', 1),
 ('собственный', 1),
 ('подержать', 1),
 ('со', 1),
 ('подержанный', 1),
 ('профильный', 1),
 ('сдача', 1),
 ('ремонт', 1)]

### Вывод

После лемматизации мы можем явно проследить основные причины взятия кредита - *недвижимость, автомобиль, образование, свадьбаб недвижимость*.

### Категоризация данных

Для лучшего представления о выборке мы можем выделить несколько категорий:

* По возрасту
* Уровень образовани
* Семейное положение
* Уровень дохода
* Цель получения

По данным таблицам мы сможем увидеть распределение клиентов по категориям. Это важно при установлении зависимостей.

Проведём категоризацию по *возрасту* клиентов. Присвоим клиентом следующие категории:

* Взрослые (от 18 до 64)
* Пенсионеры (более 64 лет)

Вариант с возрастом менее 18 лет не рассматриваем, так как кредиты не выдаются несовершеннолетним. 


In [34]:
def age_group(age):
        """
        Возвращает возврастную группу по значению возраста age, используя правила:
        - 'взрослые' при значениии age более 18 и менее 64, включая 64
        - 'пенсионеры' во всех остальных случаях
        """
        if age <= 64:
                return 'взрослые'
        return 'пенсионеры'

In [35]:
data['age_group'] = data['dob_years'].apply(age_group)

In [36]:
data.groupby('age_group').mean()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
age_group,,,,,,,
взрослые,0.559585,50321.914234,42.446960,0.812179,0.967711,0.082032,151732.904732
пенсионеры,0.064516,285781.372636,67.431591,0.933259,1.083426,0.054505,122509.890990


Исходя из данной выборки можно сказать, что у пенсионеров средний доход, чем у взрослых клиентов, но процент образованных людей - выше. Если посмотреть на средние значения по задолжностям кредита, то видно, что пенсионеры чаще выплачивают кредит в сроки. 

Рассмотрим следующую выборку - по образованию.

In [37]:
data.groupby('education').mean().sort_values(by = 'education_id')

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
education,,,,,,,
высшее,0.569011,37992.405133,40.947529,0.0,1.026616,0.052852,185718.957224
среднее,0.530756,68688.220705,44.713451,1.0,0.929298,0.089542,138175.111272
неоконченное высшее,0.513441,18740.434140,34.774194,2.0,1.420699,0.091398,164697.768817
начальное,0.482270,120633.748227,47.691489,3.0,1.113475,0.109929,122313.691489
ученая степень,0.666667,121323.333333,51.166667,4.0,1.166667,0.000000,174749.833333


По данной таблице сразу можно выделить несколько зависимостей:
* Чем выше уровень образования, тем меньше возникает проблем с выплатой кредита. 
* Выше уровень образования - выше средняя ЗП
* Много людей в возрасте не имеют среднего образования. 

Рассмотрим следующую категорию:

In [38]:
data.groupby('family_status').mean().sort_values(by = 'family_status_id')

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
family_status,,,,,,,
женат / замужем,0.638207,56986.170598,43.730937,0.805250,0.0,0.075202,152123.240065
гражданский брак,0.510175,52216.462533,42.341633,0.837683,1.0,0.092890,149185.107254
вдовец / вдова,0.223958,185286.957292,56.732292,0.936458,2.0,0.065625,129435.354167
в разводе,0.457741,62366.156485,45.869456,0.797490,3.0,0.071130,154006.209205
не женат / не замужем,0.286527,42252.849271,38.608247,0.807323,4.0,0.097405,151102.744401


По аналогии с предыдущему таблицами можно увидеть зависимости. К примеру:

* У людей в браке (официальном / гражданском) больше детей;
* Люди в браке старше не женатых / не замужних. У вдов / вдовцов средний возраст заметно выше, чем у остальных категорий;
* У вдовцов / вдов выше уровень образования. Вероятно, в связи с возрастом.


Проанализиуем уровень дохода клиентов. Для этого посмотрим минимальное / максимальное / среднее / медианное значение для таблицы *total_income.*

In [39]:
print(f"min = {data['total_income'].min()}")
print(f"max = {data['total_income'].max()}")
print(f"mean = {data['total_income'].mean()}")
print(f"median = {data['total_income'].median()}")

min = 0
max = 2265604
mean = 150512.39419279908
median = 135514.0


Разделим уровень доходов на 4 категории:

* до 50000
* 50000-80000
* 80000-150000
* 150000-2500000

Посмотрим распределение по данным категориям. 

In [41]:
def income_cat(row):

    if row < 50000:
        return  'до 50000'
    elif 50000 < row < 80000:
        return  '50000-80000'
    elif 80000 < row < 150000:
        return  '80000-150000'
    elif 150000 < row < 2500000:
        return  '150000-2500000'
        
data['income_category'] = data['total_income'].apply(income_cat)

In [42]:
data.groupby(by = 'income_category').mean().sort_values(by = 'total_income')

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
income_category,,,,,,,
до 50000,0.534171,26088.035350,44.701100,0.830715,0.939906,0.075805,6120.491359
50000-80000,0.451681,119580.087710,46.111870,0.947479,0.956408,0.079307,67191.070378
80000-150000,0.550501,71589.298188,43.442403,0.875301,1.006590,0.085034,114919.061462
150000-2500000,0.548345,47457.042683,42.652657,0.736607,0.955684,0.079051,238397.038545


Присвоим каждому клиенту свою категорию цели кредита. Для этого используем ранее полученную информацию в пункте "Лемматизация". В таблице было выявлено 5 категорий:

* жилье
* автомобиль
* образование
* свадьба
* недвижимость

Проверим столбец *purpose* и распределим строки по этим категориям. Сохраним категориии в переменную *lemmas_cat*. 

In [43]:
lemmas_cat = ['жилье', 'автомобиль', 'образование', 'свадьба', 'недвижимость']


Создадим функцию *purpose_cat*, в которой проверим принадлежность цели к указанному классу, лемматизируя значение в столбце *purpose*. 

In [44]:
def purpose_cat(row):
    if lemmas_cat[0] in m.lemmatize(row):
        return lemmas_cat[0]
    elif lemmas_cat[1] in m.lemmatize(row):
        return lemmas_cat[1]
    elif lemmas_cat[2] in m.lemmatize(row):
        return lemmas_cat[2]
    elif lemmas_cat[3] in m.lemmatize(row):
        return lemmas_cat[3]
    elif lemmas_cat[4] in m.lemmatize(row):
        return lemmas_cat[4]
    return 'undefined'



Добавим дополнительную колонку *purpose_category*, в которой и будет указан класс. 

In [45]:
data['purpose_category'] = data['purpose'].apply(purpose_cat)

In [41]:
data.groupby(by='purpose_category').mean()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
purpose_category,,,,,,,
автомобиль,0.533488,2018.929548,43.692700,0.820626,0.988413,0.093395,151117.583778
жилье,0.547060,2009.926671,43.165214,0.809524,0.959759,0.068858,153335.206573
недвижимость,0.532276,2050.677399,43.509188,0.819538,0.973771,0.074446,151148.479661
образование,0.548235,2010.020885,43.614620,0.827946,0.951765,0.091994,147067.226504
свадьба,0.535349,2046.270869,43.474872,0.801107,1.000000,0.079216,148199.223595


### Вывод

После построения таблиц с выделеными категориями клиентов можно отследить завимости, которые могут понадобиться при дальнейшем построении гипотез. 

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [46]:
data.pivot_table('debt', index = 'children')

,debt
children,
-1,0.021277
0,0.075129
1,0.092154
2,0.094404
3,0.081818
4,0.097561
5,0.000000
20,0.105263


### Вывод

Зависимость просматривается. Чем больше детей, тем чаще возникают проблемы с кредитом. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [47]:
data.pivot_table('debt', 'family_status')

,debt
family_status,
в разводе,0.071130
вдовец / вдова,0.065625
гражданский брак,0.092890
женат / замужем,0.075202
не женат / не замужем,0.097405


### Вывод

Зависимость просматривается. 
* Люди, которые не состоят в официальном браке, чаще имеют проблемы с выплатой кредита. 
* Если человек состоял / состоит в браке, то проблемы с выплатой наблюдаются реже. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [44]:
data.pivot_table('debt', 'income_category')

,debt
income_category,
150000-2500000,0.079051
50000-80000,0.079307
80000-150000,0.085034
до 50000,0.075805


### Вывод

Для отслеживания зависимости потребуются дополнительные данные. По данной выборке какой-либо зависимости не наблюдается. Есть небольшое повышение среднего значения для категории людей с заработком 80000-150000, но оно не значительное. 

- Как разные цели кредита влияют на его возврат в срок?

In [48]:
data.pivot_table('debt', 'purpose_category')

,debt
purpose_category,
автомобиль,0.093395
жилье,0.068858
недвижимость,0.074446
образование,0.091994
свадьба,0.079216


### Вывод

Наблюдается зависимость. Кредиты на такие категории, как *автомобиль, образование* чаще вызывают проблемы с выплатой кредита. 

### Шаг 4. Общий вывод

При проведении анализа были выявлены пропущенные данные, различные типы значений в столбцах, а также дублированные записи. 
В процессе очистки были обнаруженны завышенные значения в столбце с опытом работы, которые были приведены к медиане. 

Проводя выделения лемм было установлено, что много записей имели общую цель кредита, но различную запись. В результате, для всех клиентом были назначены категории, в соответствии с их целями. 

В итоге работы были выявлены следующие зависимости:

* Чем больше детей, тем реже выплачивают кредит в срок
* Люди, состоящие / состоявшие в официальном браке чаще выплачивают кредит в срок
* Нет зависимости от уровня дохода и возвратом кредита в срок. Наблюдается некоторое повышение среднего значения для категории 80 тыс - 150 тыс, однако оно не значительное. Вероятно, потребуется больше данных для выявления зависимости.
* Своевременность выплаты кредита зависит от целей. 